## Workshop 2: The Pooling Problem

### 1. Installing and Importing Packages 

We first need to pull in all the packages we will be using. Pyomo is a Python-based, open-source optimization modelling language with a diverse set of optimization capabilities. For more information, see the Pyomo [documentation](https://pyomo.readthedocs.io/en/stable/).

In [2]:
# Only run once at the start
!pip install -q pyomo
!pip install -q matplotlib

In [3]:
import matplotlib.pyplot as plt
from pyomo.environ import *
import numpy as np
import pandas as pd
from ipywidgets import FloatSlider, interact
import platform

# Solver setup for Windows or Linux
def setup_solver():
    os_name = platform.system()
    if os_name == "Windows":
        return "solver/ipopt.exe", "solver/cbc.exe", "solver/ampl.mswin64/bonmin.exe"
    elif os_name == "Linux":
        !chmod +x "solver/ipopt", "solver/cbc", "solver/bonmin"
        return "solver/ipopt", "solver/cbc", "solver/bonmin"

ipopt_executable, cbc_executable, bonmin_executable = setup_solver()

### 2. Pooling Milk for Blending and Distribution

![image.png](images\milk-pooling_simple.png)


The milk distributor blends supplies from the two local farms to meet customer requirements. Let $L$ designate the set of local suppliers, and let $C$ designate the set of customers. The decision variable $x_{l,c}$ is the amount of milk from local supplier $l\in L$ mixed into the blend for customer $c\in C$.

The distributor’s objectives is to maximize profit:

\begin{align*}
\text{Profit} & = \sum_{(l,c)\ \in\ L \times C} (\text{price}_c - \text{cost}_l) x_{l,c}
\end{align*}
 
where the notation $(l,c) \in L \times C$ indicates a summation over the cross-product of two sets. A useful interpretation is that $(l,c) \in L \times C$ describes all ways of delivering milk from $l$ to $c$. Each term ($\text{price}_{c} - \text{price}_{l}$) is then the profit earned by delivering one unit of milk from $l \in L$ to $c \in C$.

The amount of milk delivered to each customer can not exceed customer demand.

\begin{align*}
\sum_{l\in L} x_{l, c} & \leq \text{demand}_{c} & \forall c\in C
\end{align*}
 		
The milk blend delivered to each customer  must meet the minimum product quality requirement for milk fat. Assuming linear blending, the model becomes:

\begin{align*}
\sum_{(l,c)\ \in\ L \times C} \text{conc}_{l} x_{l,c} & \geq \text{conc}_{c} \sum_{l\in L} x_{l, c} & \forall c \in C
\end{align*}	

In [11]:
customers = pd.DataFrame({
    "Customer A": {"fat": 0.0445, "price": 52.0, "demand": 6000.0},
    "Customer B": {"fat": 0.030, "price": 48.0, "demand": 2500.0}
}).T

suppliers = pd.DataFrame({
    "Farm A": {"fat": 0.045, "cost": 45.0, "location": "local"},
    "Farm B": {"fat": 0.030, "cost": 42.0, "location": "local"},
    "Farm C": {"fat": 0.033, "cost": 37.0, "location": "remote"},
    "Farm D": {"fat": 0.050, "cost": 45.0, "location": "remote"}},
    ).T

local_suppliers = suppliers[suppliers["location"]=="local"]
remote_suppliers = suppliers[suppliers["location"]=="remote"]

print("\nCustomers")
print(customers)

print("\nLocal Suppliers")
print(local_suppliers)

print("\nRemote Suppliers")
print(remote_suppliers)


Customers
               fat  price  demand
Customer A  0.0445   52.0  6000.0
Customer B  0.0300   48.0  2500.0

Local Suppliers
          fat  cost location
Farm A  0.045  45.0    local
Farm B   0.03  42.0    local

Remote Suppliers
          fat  cost location
Farm C  0.033  37.0   remote
Farm D   0.05  45.0   remote


#### a. Option 1 (MILP): Current Setup - Two Local Milk Farms, Two Customers

In [22]:
# Define "fat" as the quality component for milk blending
q = "fat"

# Initialize the optimization model
model = ConcreteModel()

# Define the sets for suppliers and customers based on data indices
model.L = Set(initialize=local_suppliers.index)  # Set of local suppliers
model.C = Set(initialize=customers.index)  # Set of customers
model.L2C = (
    model.L * model.C
)  # Cross-product set representing all (supplier, customer) pairs

# Define the quality component set, which currently includes only "fat"
model.Q = Set(initialize=["fat"])

# Define the decision variables for the amount of milk from each supplier to each customer
model.x = Var(model.L * model.C, domain=NonNegativeReals)  # Flow rate variables


# Objective Function: Maximize profit by choosing flow rates optimally
@model.Objective(sense=maximize)
def profit(model):
    # Profit calculation: Sum over all (supplier, customer) pairs
    return sum(
        model.x[l, c] * (customers.loc[c, "price"] - suppliers.loc[l, "cost"])
        for l, c in model.L2C
    )


# Demand Constraints: Ensure supply to each customer does not exceed their demand
@model.Constraint(model.C)
def demand(model, c):
    # Sum of milk delivered to customer c from all suppliers should not exceed customer demand
    return sum(model.x[l, c] for l in model.L) <= customers.loc[c, "demand"]


# Quality Constraints: Ensure minimum fat content requirement in the milk blend for each customer
@model.Constraint(model.C)
def quality(model, c):
    # Sum of the fat contribution from all suppliers should meet the customer's quality requirement
    return (
        sum(model.x[l, c] * suppliers.loc[l, q] for l in model.L)
        >= sum(model.x[l, c] for l in model.L) * customers.loc[c, q]
    )


# Solve the optimization model using Bonmin (or other suitable solver)
SolverFactory("cbc", executable=cbc_executable).solve(model)

# Display the total profit from the optimized solution
print(f"\nTotal Profit = £{model.profit():.0f}\n")

# Prepare a DataFrame to display the flow rate results in an organized table
# Create a table of the flow rates from each supplier to each customer
X = pd.DataFrame(
    [[l, c, round(model.x[l, c](), 1)] for l, c in model.L2C],
    columns=["supplier", "customer", "blend"],
)
X = X.pivot_table(index="customer", columns="supplier", values="blend")

# Calculate and display the average fat content and total amount of milk delivered to each customer
X["fat"] = sum(X.loc[:, l] * suppliers.loc[l, "fat"] for l in model.L) / X.sum(axis=1)
X["Total"] = X.sum(axis=1)
print(X)


Total Profit = £57600

supplier    Farm A  Farm B     fat      Total
customer                                     
Customer A  5800.0   200.0  0.0445  6000.0445
Customer B     0.0  2500.0  0.0300  2500.0300


#### b. Option 2 (MINLP): Potential New Setup - Four Milk Farms, Two Customers

![image.png](images\milk-pooling.dio.png)

The distributor wants to see if additional profit can be made by purchasing raw milk two remote farms. This would require purchasing and operating a separate delivery truck. Blending is also not possible in the remote locations.

The question is whether the one existing truck could transport a pool of raw milk from remote farms C and D that could be blended with raw milk from local farms A and B to meet customer requirements. The profit potential may be reduced due to pooling, but it may still be a better option than adding additional operating expense.

The pooling problem is famous, and there are a number of formulations used to model it. We will use a version of the "p-parameterisation" where composition of the pool is a decision variable. Here that variable will be called $f$.

Other decision variables are also needed. Decision variables $y_c$ refer to the amount of the pool used in the blend delivered to customer $c\in C$. Variables $z_r$ ar the amount of raw milk purchased from remote farm $r$ and included in the pool.

#### Updated Problem Formulation 

In the objective function, there are new terms for the cost of raw milk included in the pool, and customer revenue earned from use of the pool. 
$$
\begin{align*}
\text{Profit} & = \sum_{(l,c)\ \in\ L \times C} (\text{price}_c - \text{cost}_l) x_{l,c}
+ \sum_{c\in C} \text{price}_c y_{c} - \sum_{r\in R} \text{cost}_r z_{r}
\end{align*}
$$

The product delivered to each customer can not exceed customer demand.

$$
\begin{align*}
\sum_{l\in L} x_{l, c} + y_{c} & \leq \text{demand}_{c} & \forall c\in C
\end{align*}
$$

Incoming and outgoing flows to the pool must balance.

$$
\begin{align*}
\sum_{r\in R}z_{r} & = \sum_{c\in C} y_{c} \\
\end{align*}
$$

The milk fat composition of the pool, $f$, is given by averaging contributions from the remote farms.

$$
\begin{align*}
\sum_{r\in R}\text{conc}_{r} z_{r}  & = \underbrace{f \sum_{c\in C} y_{c}}_{\text{bilinear}}
\end{align*}
$$

Finally, the minimum milk fat required by each customer $c\in C$ satisfies a blending constraint.

$$
\begin{align*}
\underbrace{f y_{c}}_{\text{bilinear}}  + \sum_{(l,c)\ \in\ L \times C} x_{l,c} \text{conc}_{l} 
& \geq \text{conc}_{c} (\sum_{l\in L} x_{l, c} + y_{c})
& \forall c \in C
\end{align*}
$$

The last two constraints include bilinear terms from the project of decision variable $f$ with decision variables $y_c$ for all $c\in C$. 


In [24]:
# Create the Pyomo model
m = ConcreteModel()

# Define sets for suppliers and customers
m.L = Set(initialize=local_suppliers.index)   # Local suppliers
m.R = Set(initialize=remote_suppliers.index)  # Remote suppliers
m.C = Set(initialize=customers.index)         # Customers

# Define decision variables
m.x = Var(m.L * m.C, domain=NonNegativeReals)             # Flow rates from local suppliers to customers
m.y = Var(m.C, domain=NonNegativeReals)                   # Flow rates from the pool to customers
m.z = Var(m.R, domain=NonNegativeReals)                   # Flow rates from remote suppliers to the pool
m.f = Var(bounds=(0.03, 0.051), domain=NonNegativeReals)  # Pool composition of fat content

# Objective function: Maximize profit
def profit_rule(m):
    # Profit from local suppliers to customers
    local_profit = sum(m.x[l, c] * (customers.loc[c, "price"] - suppliers.loc[l, "cost"]) for l, c in m.L * m.C)
    
    # Profit from pooled supply to customers
    pool_profit = sum(m.y[c] * customers.loc[c, "price"] for c in m.C)
    
    # Cost of remote suppliers contributing to the pool
    remote_cost = sum(m.z[r] * suppliers.loc[r, "cost"] for r in m.R)
    return local_profit + pool_profit - remote_cost

m.profit = Objective(rule=profit_rule, sense=maximize)

# Customer demand constraint
def customer_demand_rule(m, c):
    # Total supply to each customer should not exceed their demand
    return sum(m.x[l, c] for l in m.L) + m.y[c] <= customers.loc[c, "demand"]

m.customer_demand = Constraint(m.C, rule=customer_demand_rule)

# Pool balance constraint
def pool_balance_rule(m):
    # Ensure total inflow to the pool equals the total outflow to customers
    return sum(m.z[r] for r in m.R) == sum(m.y[c] for c in m.C)

m.pool_balance = Constraint(rule=pool_balance_rule)

# Pool quality constraint (nonlinear)
def pool_quality_rule(m):
    # Balance fat content in the pool, considering fat concentration and total flow rate
    return sum(suppliers.loc[r, "fat"] * m.z[r] for r in m.R) == m.f * sum(m.y[c] for c in m.C)

m.pool_quality = Constraint(rule=pool_quality_rule)

# Customer quality constraint (nonlinear)
def customer_quality_rule(m, c):
    # Ensure the fat content requirement for each customer is met by blending pool and local supplies
    return m.f * m.y[c] + sum(suppliers.loc[l, "fat"] * m.x[l, c] for l in m.L) \
         >= customers.loc[c, "fat"] * (sum(m.x[l, c] for l in m.L) + m.y[c])

m.customer_quality = Constraint(m.C, rule=customer_quality_rule)

# Solve the model using the IPOPT solver (if nonlinear)
solver = SolverFactory('bonmin', executable=bonmin_executable)
results = solver.solve(m)

# Access and print the optimized profit
optimal_profit = value(m.profit)
print(f"Optimal Profit: £{optimal_profit:.0f}")


Optimal Profit: £71500
